### 1. Создание признакового пространства 

Алгоритмы машинного обучения не могут напрямую работать с сырым текстом, поэтому необходимо конвертировать текст в наборы цифр (векторы). Это называется извлечением признаков.

##### Мешок слов
– это популярная и простая техника извлечения признаков, используемая при работе с текстом. Она описывает вхождения каждого слова в текст.

Чтобы использовать модель, нам нужно:

- Определить словарь известных слов (токенов).
- Выбрать степень присутствия известных слов.

Любая информация о порядке или структуре слов игнорируется. Вот почему это называется МЕШКОМ слов. Эта модель пытается понять, встречается ли знакомое слово в документе, но не знает, где именно оно встречается.

Интуиция подсказывает, что схожие документы имеют схожее содержимое. Также, благодаря содержимому, мы можем узнать кое-что о смысле документа.

Пример:
Рассмотрим шаги создания этой модели. Мы используем только 4 предложения, чтобы понять, как работает модель. В реальной жизни вы столкнетесь с бОльшими объемами данных.

In [ ]:
documents = ["I like this movie, it's it's it's funny.", 'I hate this movie.', 'This was awesome! I like it.', 'Nice one. I love it.']

Определяем словарь и создаем векторы документа. Соберем все уникальные слова из 4 загруженных предложений, игнорируя регистр, пунктуацию и односимвольные токены. Это и будет наш словарь (известные слова).

Для создания словаря можно использовать класс CountVectorizer из библиотеки sklearn.

In [ ]:
import warnings
warnings.filterwarnings("ignore")

In [ ]:
documents

["I like this movie, it's it's it's funny.",
 'I hate this movie.',
 'This was awesome! I like it.',
 'Nice one. I love it.']

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
import pandas as pd

count_vectorizer = CountVectorizer(ngram_range=(1, 2), analyzer='word', binary=False,)

# Создаем the Bag-of-Words модель
bag_of_words = count_vectorizer.fit_transform(documents)

# Отобразим Bag-of-Words модель как DataFrame
feature_names = count_vectorizer.get_feature_names()
pd.DataFrame(bag_of_words.toarray(), columns = feature_names)

,awesome,awesome like,funny,hate,hate this,it,it funny,it it,like,like it,...,movie it,nice,nice one,one,one love,this,this movie,this was,was,was awesome
0,0,0,1,0,0,1,1,1,1,0,...,1,0,0,0,0,1,1,0,0,0
1,0,0,0,1,1,0,0,0,0,0,...,0,0,0,0,0,1,1,0,0,0
2,1,1,0,0,0,1,0,0,1,1,...,0,0,0,0,0,1,0,1,1,1
3,0,0,0,0,0,1,0,0,0,0,...,0,1,1,1,1,0,0,0,0,0


Когда размер словаря увеличивается, вектор документа тоже растет. В примере выше, длина вектора равна количеству известных слов.

В некоторых случаях, у нас может быть неимоверно большой объем данных и тогда вектор может состоять из тысяч или миллионов элементов. Более того, каждый документ может содержать лишь малую часть слов из словаря.

Как следствие, в векторном представлении будет много нулей. Векторы с большим количеством нулей называются разреженным векторами (sparse vectors), они требуют больше памяти и вычислительных ресурсов. Частично эту проблему можно реить хорошей предобработкой.


##### Мешок N-грамм

Другой, более сложный способ создания словаря – использовать сгруппированные слова. Это изменит размер словаря и даст мешку слов больше деталей о документе. Такой подход называется «N-грамма».

N-грамма это последовательность каких-либо сущностей (слов, букв, чисел, цифр и т.д.). В контексте языковых корпусов, под N-граммой обычно понимают последовательность слов. Юниграмма это одно слово, биграмма это последовательность двух слов, триграмма – три слова и так далее. Цифра N обозначает, сколько сгруппированных слов входит в N-грамму. В модель попадают не все возможные N-граммы, а только те, что фигурируют в корпусе.

Пример:
Рассмотрим такое предложение:The office building is open today

Вот его биграммы:

- the office
- office building
- building is
- is open
- open today

Как видно, мешок биграмм – это более действенный подход, чем мешок слов.

In [ ]:
BPE
"I like this movie, it's funny. I hate this movie. This was awesome! I like it. Nice one. I love it."

nn - t1
uy - t2
t1u - t3

In [ ]:
from nltk.util import ngrams

text = "I like this movie, it's funny. I hate this movie. This was awesome! I like it. Nice one. I love it."
tokenized = text.split()
bigrams = ngrams(tokenized, 2)
list(bigrams)


[('I', 'like'),
 ('like', 'this'),
 ('this', 'movie,'),
 ('movie,', "it's"),
 ("it's", 'funny.'),
 ('funny.', 'I'),
 ('I', 'hate'),
 ('hate', 'this'),
 ('this', 'movie.'),
 ('movie.', 'This'),
 ('This', 'was'),
 ('was', 'awesome!'),
 ('awesome!', 'I'),
 ('I', 'like'),
 ('like', 'it.'),
 ('it.', 'Nice'),
 ('Nice', 'one.'),
 ('one.', 'I'),
 ('I', 'love'),
 ('love', 'it.')]

##### TF-IDF

У частотного скоринга есть проблема: слова с наибольшей частотностью имеют, соответственно, наибольшую оценку. В этих словах может быть не так много информационного выигрыша для модели, как в менее частых словах. Один из способов исправить ситуацию – понижать оценку слова, которое часто встречается во всех схожих документах. Это называется TF-IDF.

TF-IDF (сокращение от term frequency — inverse document frequency) – это статистическая мера для оценки важности слова в документе, который является частью коллекции или корпуса.

Скоринг по TF-IDF растет пропорционально частоте появления слова в документе, но это компенсируется количеством документов, содержащих это слово.

<img src='image/tf_idf.PNG'>

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd

document = ["I like this movie, it's funny funny.",
            'I hate this movie.', 
            'This was awesome! I like it.', 
            'Nice one. I love it.']
tfidf_vectorizer = TfidfVectorizer()
values = tfidf_vectorizer.fit_transform(document)

# Show the Model as a pandas DataFrame
feature_names = tfidf_vectorizer.get_feature_names()
pd.DataFrame(values.toarray(), columns = feature_names)

,awesome,funny,hate,it,like,love,movie,nice,one,this,was
0,0.000000,0.812578,0.000000,0.259329,0.320323,0.000000,0.320323,0.000000,0.000000,0.259329,0.000000
1,0.000000,0.000000,0.702035,0.000000,0.000000,0.000000,0.553492,0.000000,0.000000,0.448100,0.000000
2,0.539445,0.000000,0.000000,0.344321,0.425305,0.000000,0.000000,0.000000,0.000000,0.344321,0.539445
3,0.000000,0.000000,0.000000,0.345783,0.000000,0.541736,0.000000,0.541736,0.541736,0.000000,0.000000


In [ ]:
tfidf_vectorizer.idf_

array([1.91629073, 1.91629073, 1.91629073, 1.22314355, 1.51082562,
       1.91629073, 1.51082562, 1.91629073, 1.91629073, 1.22314355,
       1.91629073])

##### HashingVectorizer

Подсчеты и частоты могут быть очень полезны, но одним из ограничений этих методов является то, что словарный запас может стать очень большим. Это, в свою очередь, потребует больших векторов для кодирования документов и налагает большие требования к памяти и замедляет алгоритмы.

Умный обходной путь - использовать односторонний хэш слов, чтобы преобразовать их в целые числа. Умная часть заключается в том, что словарь не требуется, и вы можете выбрать произвольный длинный вектор фиксированной длины. Недостатком является то, что хеш является односторонней функцией, поэтому нет способа преобразовать кодировку обратно в слово (что может не иметь значения для многих контролируемых задач обучения).HashingVectorizer класс реализует этот подход, который можно использовать для последовательного хеширования слов, а затем для токенизации и кодирования документов по мере необходимости.

Пример ниже демонстрирует HashingVectorizer для кодирования одного документа.

In [ ]:
from sklearn.feature_extraction.text import HashingVectorizer

document = ["I like this movie, it's funny.", 'I hate this movie.', 'This was awesome! I like it.', 'Nice one. I love it.']
vectorizer = HashingVectorizer(n_features=2**4,)
values = vectorizer.fit_transform(document)
print(values.shape)
print(values.toarray())

(4, 16)
[[ 0.37796447  0.          0.          0.          0.          0.
   0.         -0.37796447  0.          0.         -0.37796447  0.
   0.          0.          0.75592895  0.        ]
 [ 0.          0.          0.         -0.57735027  0.          0.
   0.          0.          0.          0.         -0.57735027  0.
   0.          0.          0.57735027  0.        ]
 [ 0.          0.4472136   0.          0.          0.          0.
   0.          0.          0.          0.          0.          0.
   0.          0.          0.89442719  0.        ]
 [ 0.          0.          0.          0.         -0.5         0.
   0.          0.          0.         -0.5         0.          0.
   0.          0.5         0.5         0.        ]]


Выполнение примера кодирует образец документа как разреженный массив из 16 элементов. Значения закодированного документа соответствуют нормализованному количеству слов по умолчанию в диапазоне от -1 до 1, но могут быть сделаны простые целочисленные счетчики путем изменения конфигурации по умолчанию.

##### Для чего нужны Vectorizers?

Алгоритмы машинного обучения не могут напрямую работать с сырым текстом, поэтому необходимо конвертировать текст в наборы цифр (векторы). Уже с векторным представлением можно производить классификацию, к примеру.

In [ ]:
# Загружаем данные
data = open('corpus').read()
labels, texts = [], []
for i, line in enumerate(data.split("\n")):
    content = line.split()
    labels.append(content[0])
    texts.append(" ".join(content[1:]))

# создаем df
trainDF = pd.DataFrame()
trainDF['text'] = texts
trainDF['label'] = labels
trainDF.head(5)

,text,label
0,Stuning even for the non-gamer: This sound tra...,__label__2
1,The best soundtrack ever to anything.: I'm rea...,__label__2
2,Amazing!: This soundtrack is my favorite music...,__label__2
3,Excellent Soundtrack: I truly like this soundt...,__label__2
4,"Remember, Pull Your Jaw Off The Floor After He...",__label__2


In [ ]:
from sklearn import model_selection, preprocessing, linear_model

In [ ]:
linear_model.LogisticRegression()

In [ ]:
from sklearn import model_selection, preprocessing, linear_model,
train_x, valid_x, train_y, valid_y = model_selection.train_test_split(trainDF['text'], trainDF['label'])

# labelEncode целевую переменную
encoder = preprocessing.LabelEncoder()
train_y = encoder.fit_transform(train_y)
valid_y = encoder.fit_transform(valid_y)


count_vect = CountVectorizer(analyzer='word', token_pattern=r'\w{1,}')
count_vect.fit(trainDF['text'])

xtrain_count =  count_vect.transform(train_x)
xvalid_count =  count_vect.transform(valid_x)

classifier = linear_model.LogisticRegression()
classifier.fit(xtrain_count, train_y)
predictions = classifier.predict(xvalid_count)
predictions

array([1, 0, 0, ..., 0, 1, 0])

In [ ]:
from sklearn.metrics import accuracy_score

In [ ]:
accuracy_score(valid_y, predictionsictions)

0.8624

In [ ]:
trainDF['label'].value_counts()

__label__1    5097
__label__2    4903
Name: label, dtype: int64

### Векторные представления слов (word embeddings)

Векторное представление слова (word embedding) — вещественный вектор в пространстве с фиксированной невысокой размерностью.

                                            Пример векторных представлений слов (2D t-SNE)
<img src='image/2D_tsne.PNG'>

Зачем нужны Word embeddings?
Сжатые векторные представления слов
1. полезны сами по себе, например, для поиска
синонимов или опечаток в поисковых запросах.
2. используются в качестве признаков для решения
самых различных задач: выявление именованных сущностей, тэгирование частей речи, машинный перевод, кластеризация документов, ранжирование документов, анализ тональности текста.

#### word2vec

Мера семантической близости — мера близости, предназначенная для количественной оценки семантической схожести слов. Такая мера показывает высокие значения для пар слов, находящихся в семантических отношениях (синонимия, ассоциативность и т.д.), и нулевые значения для всех остальных пар.

word2vec - алгоритм для получения векторных представлений слов. Подход основан на важной гипотезе, которую в науке принято называть гипотезой локальности — “слова, которые встречаются в одинаковых окружениях, имеют близкие значения”. Близость в данном случае понимается очень широко, как то, что рядом могут стоять только сочетающиеся слова. Например, для нас привычно словосочетание "заводной будильник". А сказать “заводной апельсин” мы не можем* — эти слова не сочетаются.

##### Алгоритм word2vec
Мы будем предсказывать вероятность слова по его окружению (контексту). То есть мы будем учить такие вектора слов, чтобы вероятность, присваиваемая моделью слову была близка к вероятности встретить это слово в этом окружении в реальном тексте.





<img src='image/w2v_formula.PNG'>

Здесь W0 — вектор целевого слова, Wc — это некоторый вектор контекста, вычисленный (например, путем усреднения) из векторов окружающих нужное слово других слов. А S — это функция, которая двум векторам сопоставляет одно число. Например, это может быть косинусное расстояние.

Процесс тренировки устроен следующим образом: мы берем последовательно (2k+1) слов, слово в центре является тем словом, которое должно быть предсказано. А окружающие слова являются контекстом длины по k с каждой стороны. Каждому слову в нашей модели сопоставлен уникальный вектор, который мы меняем в процессе обучения нашей модели. В целом, этот подход называется CBOW — continuous bag of words, continuous потому, что мы скармливаем нашей модели последовательно наборы слов из текста, a BoW потому что порядок слов в контексте не важен.
<img src='image/CBOW_.png'>
Другой подход skip-gram — прямо противоположный CBOW, то есть “словосочетание с пропуском”. Мы пытаемся из данного нам слова угадать его контекст (точнее вектор контекста). В остальном модель не претерпевает изменений.
<img src='image/skipgram.png'>

Что стоит отметить: хотя в модель не заложено явно никакой семантики, а только статистические свойства корпусов текстов, оказывается, что натренированная модель word2vec может улавливать некоторые семантические свойства слов. Классический пример:

<img src='image/word_embeddings.PNG'>

Слово "мужчина" относится к слову "женщина" так же, как слово "дядя" к слову "тётя", что для нас совершенно естественно и понятно, но в других моделям добиться такого же соотношения векторов можно только с помощью специальных ухищрений. Здесь же — это происходит естественно из самого корпуса текстов.

#### Что мы можем попробовать сделать с векторами слов?

Мы можем делать различные синтаксические, семантические NLP задачи с векторами слов, некоторое из них уже встроены. 


In [ ]:
import gensim.downloader as api

In [ ]:
api.info()['models'].keys()

dict_keys(['fasttext-wiki-news-subwords-300', 'conceptnet-numberbatch-17-06-300', 'word2vec-ruscorpora-300', 'word2vec-google-news-300', 'glove-wiki-gigaword-50', 'glove-wiki-gigaword-100', 'glove-wiki-gigaword-200', 'glove-wiki-gigaword-300', 'glove-twitter-25', 'glove-twitter-50', 'glove-twitter-100', 'glove-twitter-200', '__testing_word2vec-matrix-synopsis'])

In [ ]:
import gensim.downloader as api

word_vectors = api.load("glove-wiki-gigaword-100")  # загрузим предтренированные вектора слов из gensim-data
# выведим слово наиболее близкое к 'woman', 'king' и далекое от 'man'
result = word_vectors.most_similar(positive=['woman', 'king'], negative=['man'])
print("{}: {:.4f}".format(*result[0]))

[==================================================] 100.0% 128.1/128.1MB downloaded
queen: 0.7699


In [ ]:
# выведем лишнее слово
print(word_vectors.doesnt_match("breakfast cereal dinner lunch".split()))

print(word_vectors.doesnt_match("black green summer brown".split()))

cereal
summer


In [ ]:
# определим схожесть между словами
similarity = word_vectors.similarity('woman', 'man')
print(similarity)

similarity = word_vectors.similarity('human', 'man')
print(similarity)

similarity = word_vectors.similarity('bee', 'man')
print(similarity)

0.8323495
0.5288512
0.21199903


In [ ]:
# найдем top-3 самых близких слов
result = word_vectors.similar_by_word("man", topn=3)
print(result)

result = word_vectors.similar_by_word("cat", topn=3)
print(result)

result = word_vectors.similar_by_word("mouth", topn=3)
print(result)

[('woman', 0.832349419593811), ('boy', 0.7914870977401733), ('one', 0.7788748741149902)]
[('dog', 0.8798074722290039), ('rabbit', 0.7424426674842834), ('cats', 0.7323004007339478)]
[('tongue', 0.7366125583648682), ('mouths', 0.687748908996582), ('ear', 0.6811771988868713)]


# Simple chat-bot example

In [ ]:
import string
from pymorphy2 import MorphAnalyzer
from stop_words import get_stop_words
import annoy
from gensim.models import Word2Vec, FastText
import pickle
import numpy as np
from tqdm import tqdm_notebook

In [ ]:
assert True

#Small preprocess of the answers

question = None
written = False

with open("prepared_answers.txt", "w") as fout:
    with open("Otvety.txt", "r") as fin:
        for line in tqdm_notebook(fin):
            if line.startswith("---"):
                written = False
                continue
            if not written and question is not None:
                fout.write(question.replace("\t", " ").strip() + "\t" + line.replace("\t", " "))
                written = True
                question = None
                continue
            if not written:
                question = line.strip()
                continue

In [ ]:
def preprocess_txt(line):
    spls = "".join(i for i in line.strip() if i not in exclude).split()
    spls = [morpher.parse(i.lower())[0].normal_form for i in spls]
    spls = [i for i in spls if i not in sw and i != ""]
    return spls

In [ ]:
assert True

# Preprocess for models fitting

sentences = []

morpher = MorphAnalyzer()
sw = set(get_stop_words("ru"))
exclude = set(string.punctuation)
c = 0

with open("Otvety.txt", "r") as fin:
    for line in tqdm_notebook(fin):
        spls = preprocess_txt(line)
        sentences.append(spls)
        c += 1
        if c > 100000:
            break

In [ ]:
sentences = [i for i in sentences if len(i) > 2]

In [ ]:
sentences[0]

['вопрос', 'тдв', 'отдыхать', 'лично', 'советовать', 'завести']

In [ ]:
modelW2V = Word2Vec(sentences=sentences, size=50, window=5, min_count=1)

In [ ]:
modelFT = FastText(sentences=sentences, size=50, min_count=1, window=5)

In [ ]:
w2v_index = annoy.AnnoyIndex(50 ,'angular')
ft_index = annoy.AnnoyIndex(50 ,'angular')

index_map = {}
counter = 0

with open("prepared_answers.txt", "r") as f:
    for line in tqdm_notebook(f):
        n_w2v = 0
        n_ft = 0
        spls = line.split("\t")
        index_map[counter] = spls[1]
        question = preprocess_txt(spls[0])
        
        vector_w2v = np.zeros(50)
        vector_ft = np.zeros(50)
        for word in question:
            if word in modelW2V:
                vector_w2v += modelW2V[word]
                n_w2v += 1
            if word in modelFT:
                vector_ft += modelFT[word]
                n_ft += 1
        if n_w2v > 0:
            vector_w2v = vector_w2v / n_w2v
        if n_ft > 0:
            vector_ft = vector_ft / n_ft
        w2v_index.add_item(counter, vector_w2v)
        ft_index.add_item(counter, vector_ft)
            
        counter += 1
        
        if counter > 100000:
            break

w2v_index.build(10)
ft_index.build(10)

True

In [ ]:
def get_response(question, index, model, index_map):
    question = preprocess_txt(question)
    vector = np.zeros(50)
    norm = 0
    for word in question:
        if word in model:
            vector += model[word]
            norm += 1
    if norm > 0:
        vector = vector / norm
    answers = index.get_nns_by_vector(vector, 3)
    return [index_map[i] for i in answers]

In [ ]:
TEXT = "тапочки какого цвета"

In [ ]:
get_response(TEXT, w2v_index, modelW2V, index_map)

NameError: ignored

In [ ]:
get_response(TEXT, ft_index, modelFT, index_map)

['[ссылка заблокирована по решению администрации проекта]. \n',
 '<p>Шартрёз (от фр.\xa0chartreuse)\xa0— оттенок жёлто-зелёного цвета. Цвет шартрёз находится примерно посередине частот видимого спектра[источник?]. </p> <p>Своим названием цвет обязан ликёру шартрёз. </p> <p>Существуют два оттенка шартрёз: </p> <ul> <li>Зелёный шартрёз #7FFF00 — схож с цветом ликёра «зелёный шартрёз», цвет которого обусловлен настоем из 130 трав, входящим в его состав (зелёный пигмент\xa0— это в основном хлорофилл).</li></ul> <ul> <li>Жёлтый шартрёз #DFFF00 \xa0— схож с цветом ликёра «жёлтый шартрёз», изготовленного с применением тех же растений, что и зелёный, но в других пропорциях. Пигмент, определяющий цвет напитка\xa0— шафран.</li></ul>. \n',
 'Калом бур, а телом бел, как Ржевский. \n']

In [ ]:
(12,3,9,0,)/